<a href="https://colab.research.google.com/github/NataliaGutman/AERONAUTICAL-PROJECT/blob/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to C:\Users\Luz
[nltk_data]     Helena\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Luz
[nltk_data]     Helena\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import requests
import pickle

# Descargar el archivo JSON desde GitHub
raw_url = "https://raw.githubusercontent.com/NataliaGutman/AERONAUTICAL-PROJECT/f4261aff6dcf8e41164f57232c1abacd783792d8/intents.json"
response = requests.get(raw_url)

if response.status_code == 200:
    datos_json = json.loads(response.text)
else:
    print("Error al recuperar datos de GitHub.")

# Procesar los datos del archivo JSON
words = []
classes = []
documents = []
ignore_words = ['?']

for intent in datos_json['intents']:
    for pattern in intent['text']:
        # Tokenizar las palabras en los patrones
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Agregar el patrón y la etiqueta de clase al documento
        documents.append((w, intent['tag']))
        # Agregar la etiqueta de clase a la lista de clases
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lematizar y eliminar duplicados
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# Crear datos de entrenamiento
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Mezclar los datos de entrenamiento
random.shuffle(training)

# Convertir datos de entrenamiento a arrays NumPy
train_x = np.array([item[0] for item in training])
train_y = np.array([item[1] for item in training])

# Construir el modelo de red neuronal
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Utilizar el optimizador SGD de TensorFlow
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Guardar el modelo y los datos de entrenamiento
model.save('chatbot_model.h5')
pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, open('training_data', 'wb'))

print("Chatbot: El modelo se ha entrenado y guardado con éxito. Ahora puedes usarlo para interactuar.")

# Definir funciones para limpiar texto y predecir respuestas
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return(np.array(bag))

def predict_class(sentence):
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints):
    tag = ints[0]['intent']
    list_of_intents = datos_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice

Epoch 1/200
3/3 [==============================] - 1s 6ms/step - loss: 1.1316 - accuracy: 0.3333
Epoch 2/200
3/3 [==============================] - 0s 5ms/step - loss: 1.1932 - accuracy: 0.4000
Epoch 3/200
3/3 [==============================] - 0s 5ms/step - loss: 1.1492 - accuracy: 0.2667
Epoch 4/200
3/3 [==============================] - 0s 6ms/step - loss: 1.0809 - accuracy: 0.4667
Epoch 5/200
3/3 [==============================] - 0s 7ms/step - loss: 1.0860 - accuracy: 0.5333
Epoch 6/200
3/3 [==============================] - 0s 4ms/step - loss: 1.0671 - accuracy: 0.5333
Epoch 7/200
3/3 [==============================] - 0s 4ms/step - loss: 1.0958 - accuracy: 0.4667
Epoch 8/200
3/3 [==============================] - 0s 5ms/step - loss: 1.0163 - accuracy: 0.5333
Epoch 9/200
3/3 [==============================] - 0s 6ms/step - loss: 0.8962 - accuracy: 0.6667
Epoch 10/200
3/3 [==============================] - 0s 6ms/step - loss: 0.9257 - accuracy: 0.4667
Epoch 11/200
3/3 [===========

c:\Users\Luz Helena\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
import tkinter
from tkinter import *

# Define the chatbot_response function
def get_response(ints):
    tag = ints[0]['intent']
    list_of_intents = datos_json['intents']
    for intent in list_of_intents:
        if intent['tag'] == tag:
            responses = intent['response']
            return random.choice(responses)

    # Si no se encuentra una coincidencia, puedes devolver una respuesta predeterminada
    return "Lo siento, no entiendo tu pregunta."

# Ejemplo de uso en chatbot_response
def chatbot_response(msg):
    ints = predict_class(msg)
    response = get_response(ints)
    return response


# Define the send function
def send():
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0", END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12))
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

# Create the main window
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

# Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial")
ChatLog.config(state=DISABLED)

# Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

# Create Button to send message
SendButton = Button(base, font=("Verdana", 12, 'bold'), text="Send", width="12", height=5, bd=0, bg="#32de97", activebackground="#3c9d9b", fg='#ffffff', command=send)

# Create the box to enter message
EntryBox = Text(base, bd=0, bg="white", width="29", height="5", font="Arial")

# Place all components on the screen
scrollbar.place(x=376, y=6, height=386)
ChatLog.place(x=6, y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

# Start the GUI event loop
base.mainloop()

1/1 [==============================] - 0s 150ms/step
